[![](https://mermaid.ink/img/pako:eNptkc1Kw0AYRV9lmHX6AlkIbdPWjStdOcliaKZNoPkhThaSFEqlJdhNQF2IEolQUYTiX6WCxZdxkvEtTJNaIzirOd89DMN3Pdi2VAJF2HWwrYE9STZBdqroc_mRnt7ywYi_L9hoqoBKZQvUULpYprM4Dceg2iUmVQq9lqd1jwXjJAr5_SR5HH7FEYsm_UKorwSfD2N29JZcTfndtQ8kVGb2ELDZq1LWk-MBj0-KN33QQGX-R-dPMZuc_-hNVOY_popT_toWSsxcWBik8zCJLtdRo4gKaJaRlcM2YhcRGz6n85vNghQoQIM4BtbVbI_eSpch1YhBZChmV5V0sNujMpTJfqZil1q7h2YbitRxiQBdW8WUSDpOGjCg2MG9g820oerUcjZDG5v7lvUrkTzeKQrMexSgY7ldbW30v0kGPk8?type=png)](https://mermaid.live/edit#pako:eNptkc1Kw0AYRV9lmHX6AlkIbdPWjStdOcliaKZNoPkhThaSFEqlJdhNQF2IEolQUYTiX6WCxZdxkvEtTJNaIzirOd89DMN3Pdi2VAJF2HWwrYE9STZBdqroc_mRnt7ywYi_L9hoqoBKZQvUULpYprM4Dceg2iUmVQq9lqd1jwXjJAr5_SR5HH7FEYsm_UKorwSfD2N29JZcTfndtQ8kVGb2ELDZq1LWk-MBj0-KN33QQGX-R-dPMZuc_-hNVOY_popT_toWSsxcWBik8zCJLtdRo4gKaJaRlcM2YhcRGz6n85vNghQoQIM4BtbVbI_eSpch1YhBZChmV5V0sNujMpTJfqZil1q7h2YbitRxiQBdW8WUSDpOGjCg2MG9g820oerUcjZDG5v7lvUrkTzeKQrMexSgY7ldbW30v0kGPk8)

In [ ]:
%%capture
!pip install google-generativeai
!pip install yfinance pandas_ta

In [ ]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd
import os
import google.generativeai as genai
from IPython.display import Markdown

In [ ]:
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [ ]:
# 股價走勢查詢函數
def get_stock_price_history(symbol, period="1mo"):
    try:
        stock = yf.Ticker(symbol)
        history = stock.history(period=period)
        return history.tail().to_dict('records')
    except Exception as e:
        return f"無法獲取 {symbol} 的歷史價格: {str(e)}"

In [ ]:
# 股票資訊查詢函數
def get_stock_info(symbol):
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        return {
            '公司名稱': info.get('longName', 'N/A'),
            '現價': info.get('currentPrice', 'N/A'),
            '市值': info.get('marketCap', 'N/A'),
            '產業': info.get('industry', 'N/A'),
            '52週高點': info.get('fiftyTwoWeekHigh', 'N/A'),
            '52週低點': info.get('fiftyTwoWeekLow', 'N/A')
        }
    except Exception as e:
        return f"無法獲取 {symbol} 的資訊: {str(e)}"

In [ ]:
# 基本財務數據查詢函數
def get_financial_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        financials = stock.financials.iloc[:, 0]
        balance_sheet = stock.balance_sheet.iloc[:, 0]
        return {
            '營收': financials.get('Total Revenue', 'N/A'),
            '淨利': financials.get('Net Income', 'N/A'),
            '總資產': balance_sheet.get('Total Assets', 'N/A'),
            '總負債': balance_sheet.get('Total Liabilities Net Minority Interest', 'N/A'),
            'ROE': stock.info.get('returnOnEquity', 'N/A'),
            '本益比': stock.info.get('trailingPE', 'N/A')
        }
    except Exception as e:
        return f"無法獲取 {symbol} 的財務數據: {str(e)}"

In [ ]:
# 技術分析函數
def get_technical_analysis(symbol, period="6mo"):
    try:
        stock = yf.Ticker(symbol)
        df = stock.history(period=period)

        # 計算技術指標
        df['RSI'] = ta.rsi(df['Close'], length=14)
        macd = ta.macd(df['Close'])
        df = pd.concat([df, macd], axis=1)
        df['SMA_20'] = ta.sma(df['Close'], length=20)
        df['SMA_60'] = ta.sma(df['Close'], length=60)
        stoch = ta.stoch(df['High'], df['Low'], df['Close'])
        df = pd.concat([df, stoch], axis=1)

        latest = df.iloc[-1]
        return {
            'indicators': {
                'RSI': round(latest['RSI'], 2),
                'MACD': round(latest['MACD_12_26_9'], 2),
                'MACD_Signal': round(latest['MACDs_12_26_9'], 2),
                'SMA_20': round(latest['SMA_20'], 2),
                'SMA_60': round(latest['SMA_60'], 2),
                'K值': round(latest['STOCHk_14_3_3'], 2),
                'D值': round(latest['STOCHd_14_3_3'], 2),
                '收盤價': round(latest['Close'], 2)
            },
            'historical_data': df.tail().to_dict('records')
        }
    except Exception as e:
        return f"無法進行技術分析: {str(e)}"

In [ ]:
# Gemini 專業分析函數
def analyze_with_gemini(query, context):
    model = genai.GenerativeModel('gemini-2.0-pro-exp-02-05')
    prompt = f"""
    你是一個專業的金融分析助手，請根據以下數據回答用戶問題：
    上下文數據：{context}
    用戶問題：{query}
    請用專業且易懂的方式進行分析，並給出投資建議。
    """
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"API請求失敗: {str(e)}"

In [ ]:
# 主交互系統
def main():
    print("歡迎使用股票市場助手（Gemini版）！")
    print("可查詢功能：即時報價/財務數據/技術分析/綜合評估")

    while True:
        symbol = input("\n請輸入股票代號 (e.g. 2330.TW) 或輸入exit退出: ")
        if symbol.lower() in ['exit', 'quit']:
            break

        # 獲取基礎數據
        price_data = get_stock_price_history(symbol)
        info_data = get_stock_info(symbol)
        financial_data = get_financial_data(symbol)
        technical_data = get_technical_analysis(symbol)

        # 構建分析上下文
        context = {
            '基本資訊': info_data,
            '財務數據': financial_data,
            '技術指標': technical_data.get('indicators', {}),
            '歷史數據': technical_data.get('historical_data', [])
        }

        # 獲取用戶問題
        query = input("請輸入您的分析問題 (e.g. 技術面分析/財務健康度/綜合評估): ")

        # 使用Gemini進行分析
        analysis = analyze_with_gemini(query, context)
        display(Markdown(f"**Gemini分析結果:**\n{analysis}"))

if __name__ == "__main__":
    main()